In [4]:
import cptdl as dl 
import cptio as cio 
import cptcore as cc 
import cptextras as ce 


import xarray as xr 
import datetime as dt 
from pathlib import Path 
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs
import numpy as np

import cartopy.feature as cartopyFeature

from PIL import Image
import os

In [54]:
import datetime
from requests import HTTPError

In [6]:
case_name = "pycpt_GTM_EPS_PRECP"  
case_directory = f'/home/guillermo/DEV/EPS/cases/{case_name}'
os.mkdir(case_directory)

#### Parameters - This cell defines the parameters of your analysis

In [7]:
MOS = 'CCA'
predictor_names = [ 
    "CCSM4.PRCP", 
    "CanSIPSIC3.PRCP",
    "GEOSS2S.PRCP",
    "SPEAR.PRCP", 
    "CFSv2.PRCP",
    
    "SEAS5.PRCP",
    "METEOFRANCE8.PRCP",
    "GLOSEA6.PRCP"
]

predictand_name = 'UCSB.PRCP' # UCSB es CHIRPS v2p0

# use dl.observations.keys() to see all options for predictand 
# and dl.hindcasts.keys() to see all options for predictors
# make sure your first_year & final_year are compatible with 
# your selections for your predictors and predictands 

In [22]:
print("DISPONIBILIDAD DE DATOS HINDCAST PARA LOS MODELOS SELECCIONADOS:")
starts = []
ends = []
for model in [pred.split(".")[0] for pred in predictor_names]:
    print("\t", model, dl.metadata[model]["hindcast_limits"], dl.metadata[model]["forecast_limits"])
    starts.append(datetime.date.fromisoformat(dl.metadata[model]["hindcast_limits"]["start"]))
    if dl.metadata[model]["hindcast_limits"]["end"] != -1:
        ends.append(datetime.date.fromisoformat(dl.metadata[model]["hindcast_limits"]["end"]))
print("Rango de datos de Hindcast:", max(starts), " - ", min(ends))

DISPONIBILIDAD DE DATOS HINDCAST PARA LOS MODELOS SELECCIONADOS:
	 CCSM4 {'start': '1982-01-01', 'end': -1} {'start': '1982-01-01', 'end': -1}
	 CanSIPSIC3 {'start': '1980-01-01', 'end': '2020-12-01'} {'start': '2021-10-01', 'end': -1}
	 GEOSS2S {'start': '1981-02-01', 'end': -1} {'start': '2017-02-01', 'end': -1}
	 SPEAR {'start': '1991-01-01', 'end': -1} {'start': '2020-12-01', 'end': -1}
	 CFSv2 {'start': '1982-01-01', 'end': -1} {'start': '2011-04-01', 'end': -1}
	 SEAS5 {'start': '1981-01-01', 'end': '2016-12-01'} {'start': '2017-09-01', 'end': -1}
	 METEOFRANCE8 {'start': '1993-01-01', 'end': '2016-12-01'} {'start': '2021-07-01', 'end': -1}
	 GLOSEA6 {'start': '1993-01-01', 'end': '2016-12-01'} {'start': '2021-03-01', 'end': -1}
Rango de datos: 1993-01-01  -  2016-12-01


23 años disponibles

In [48]:

download_args = { 
   # 'fdate':
   #   the initialization date of the model forecasts / hindcasts
   #   this field is defined by a python datetime.datetime object
   #   for example: dt.datetime(2022, 5, 1) # YYYY, MM, DD as integers
   #   The year field is only used for forecasts, otherwise ignored
   #   The day field is only used in subseasonal forecasts, otherwise ignored
   #   The month field is an integer representing a month - ie, May=5
  'fdate':  dt.datetime(2021, 4, 1), #dt.datetime(2022, 6, 1),  
    
   # 'first_year':
   #   the first year of hindcasts you want. **NOT ALL MODELS HAVE ALL YEARS**
   #   double check that your model has hindcast data for all years in [first_year, final_year]
   #   This field is defined by a python integer representing a year, ie: 1993
  'first_year': 1993,  
    
   # 'final_year':
   #   the final year of hindcasts you want. **NOT ALL MODELS HAVE ALL YEARS**
   #   double check that your model has hindcast data for all years in [first_year, final_year]
   #   This field is defined by a python integer representing a year, ie: 2016
  'final_year': 2016,  
    
   # 'predictor_extent':
   #   The geographic bounding box of the climate model data you want to download
   #   This field is defined by a python dictionary with the keys "north", "south",
   #   "east", and "west", each of which maps to a python integer representing the 
   #   edge of a bounding box. i.e., "north" will be the northernmost boundary,
   #   "south" the southernmost boundary. Example: {"north": 90, "south": 90, "east": 0, "west": 180}
  'predictor_extent': {
    'east':  -70,
    'west': -120, 
    'north': 40,
    'south': -5
  }, 
    
   # 'predictand_extent':
   #   The geographic bounding box of the observation data you want to download
   #   This field is defined by a python dictionary with the keys "north", "south",
   #   "east", and "west", each of which maps to a python integer representing the 
   #   edge of a bounding box. i.e., "north" will be the northernmost boundary,
   #   "south" the southernmost boundary. Example: {"north": 90, "south": 90, "east": 0, "west": 180}
  'predictand_extent': {
    'east':  -70,
    'west': -120, 
    'north': 40,
    'south': -5
  },

    
   # 'lead_low': 
   #   the number of months from the first of the initialization month to the center of 
   #   the first month included in the target period. Always an integer + 0.5. 
   #   this field is defined by a python floating point number 
   #   for example  a lead-1 forecast would use lead_low=1.5, if you want init=may, target=Jun-..
  'lead_low': 1.5,
    
   # 'lead_high': 
   #   the number of months from the first of the initialization month to the center of 
   #   the last month included in the target period. Always an integer + 0.5. 
   #   this field is defined by a python floating point number 
   #   for example  a forecast initialized in may, whose target period ended in Aug, 
   #   would use lead_high=3.5
  'lead_high': 3.5, 
    
   # 'target': 
   #   Mmm-Mmm indicating the months included in the target period of the forecast. 
   #   this field is defined by a python string, with two three-letter month name abbreviations 
   #   whose first letters are capitalized, and all other letters are lowercase
   #   and who are separated by a dash character. 
   #   for example, if you wanted a JJA target period, you would use 'Jun-Aug'
  'target': 'May-Jul',#'Jul-Sep',
    
   # 'filetype':
   #   the filetype to be downloaded. for now, it saves a lot of headache just to set this equal
   #   to 'cptv10.tsv' which is a boutique plain-text CPT filetype based on .tsv + metadata
  'filetype': 'cptv10.tsv'
}

cpt_args = { 
    'transform_predictand': None,  # transformation to apply to the predictand dataset - None, 'Empirical', 'Gamma'
    'tailoring': None,  # tailoring None, 'Anomaly', 'StdAnomaly', or 'SPI' (SPI only available on Gamma)
    'cca_modes': (1,3), # minimum and maximum of allowed CCA modes 
    'x_eof_modes': (1,5), # minimum and maximum of allowed X Principal Componenets 
    'y_eof_modes': (1,5), # minimum and maximum of allowed Y Principal Components 
    'validation': 'crossvalidation', # the type of validation to use - crossvalidation, retroactive, or doublecrossvalidation
    'drymask': False, #whether or not to use a drymask of -999
    'scree': True, # whether or not to save % explained variance for eof modes
    'crossvalidation_window': 5,  # number of samples to leave out in each cross-validation step 
    'synchronous_predictors': True, # whether or not we are using 'synchronous predictors'
}


force_download = False

In [49]:
#extracting domain boundaries and create house keeping
domain = download_args['predictor_extent']
e,w,n,s = domain.values()

domainFolder = f"{w}W-{e}E_to_{s}S-{n}N"

domainDir = f'{case_directory}/{domainFolder}'
os.makedirs(case_directory, exist_ok=True)
dataDir = f'{case_directory}/{domainFolder}/data'
os.makedirs(dataDir, exist_ok=True)
figDir = f'{case_directory}/{domainFolder}/figures'
os.makedirs(figDir, exist_ok=True)
outputDir = f'{case_directory}/{domainFolder}/output'
os.makedirs(outputDir, exist_ok=True)
config_file = ce.save_configuration(case_directory+'/.config', download_args, cpt_args, MOS, predictor_names, predictand_name )

#### Download Observations

In [50]:
# Deal with "Cross-year issues" where either the target season crosses Jan 1 (eg DJF), 
# or where the forecast initialization is in the calendar year before the start of the target season
# (eg JFM from Dec 1 sart)

fmon=download_args['fdate'].month
tmon1 = fmon + download_args['lead_low'] # first month of the target season
tmon2 = fmon + download_args['lead_high'] # last month of the target season
download_args_obs = download_args.copy()


# For when the target season crossing Jan 1 (eg DJF)
# (i.e., when target season starts in the same calendar year as the forecast init 
# and ends in the following calendar year)
# Here the final year of the obs dataset needs to be incremented by 1.
if tmon1 <= 12.5 and tmon2 > 12.5:
    download_args_obs['final_year'] +=1    

# For JFM, FMA .. with forecast initialization in the previous year.
# (i.e., when target season starts in the calendar year after the forecast init.)
# Here both the first and final year of the obs dataset need to be incremented by 1.
if tmon1 > 12.5: 
    download_args_obs['first_year'] +=1
    download_args_obs['final_year'] +=1 
    
print(download_args) 
print(download_args_obs)

if not os.path.exists(f'{dataDir}/{predictand_name}.nc') or force_download:
    Y = dl.download(dl.observations[predictand_name], 
                    f'{dataDir}/{predictand_name}.tsv', 
                    **download_args_obs, 
                    verbose=True, 
                    use_dlauth=False)
    Y = getattr(Y, [i for i in Y.data_vars][0])
    Y.to_netcdf(f'{dataDir}/{predictand_name}.nc')
else:
    Y = xr.open_dataset(f'{dataDir}/{predictand_name}.nc')
    Y = getattr(Y, [i for i in Y.data_vars][0])

{'fdate': datetime.datetime(2021, 4, 1, 0, 0), 'first_year': 1993, 'final_year': 2016, 'predictor_extent': {'east': -70, 'west': -120, 'north': 40, 'south': -5}, 'predictand_extent': {'east': -70, 'west': -120, 'north': 40, 'south': -5}, 'lead_low': 1.5, 'lead_high': 3.5, 'target': 'May-Jul', 'filetype': 'cptv10.tsv'}
{'fdate': datetime.datetime(2021, 4, 1, 0, 0), 'first_year': 1993, 'final_year': 2016, 'predictor_extent': {'east': -70, 'west': -120, 'north': 40, 'south': -5}, 'predictand_extent': {'east': -70, 'west': -120, 'north': 40, 'south': -5}, 'lead_low': 1.5, 'lead_high': 3.5, 'target': 'May-Jul', 'filetype': 'cptv10.tsv'}


#### Download Hindcast Data

In [51]:
# download training data 
hindcast_data = []
for model in predictor_names: 
    if not os.path.exists(f"{dataDir}/{model}.nc") or force_download:
        X = dl.download(dl.hindcasts[model], 
                        f"{dataDir}{model}.tsv", 
                        **download_args, 
                        verbose=True, 
                        use_dlauth=False)
        X = getattr(X, [i for i in X.data_vars][0])
        X.name = Y.name
        X.to_netcdf(f"{dataDir}/{model}.nc")
    else:
        X = xr.open_dataset(f"{dataDir}/{model}.nc")
        X = getattr(X, [i for i in X.data_vars][0])
        X.name = Y.name
    hindcast_data.append(X)

#### Download Forecast Data

In [55]:
# download forecast data 
forecast_data = []
for model in predictor_names: 
    if not os.path.exists(f"{dataDir}/{model}_f.nc") or force_download:
        try:
            F = dl.download(dl.forecasts[model], 
                            f"{dataDir}/{model}_f.tsv", 
                            **download_args, 
                            verbose=True, 
                            use_dlauth=False)
            F = getattr(F, [i for i in F.data_vars][0])
            F.name = Y.name
            F.to_netcdf(f"{dataDir}{model}_f.nc")
        except HTTPError as httperr:
            print(f"ERROR: No se encontró datos de forecast para el modelo {model} {httperr}")
            F = None
    else:
        F = xr.open_dataset(f"{dataDir}{model}_f.nc")
        F = getattr(F, [i for i in F.data_vars][0])
        F.name = Y.name
    forecast_data.append(F)

URL: https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.COLA-RSMAS-CCSM4/.MONTHLY/.prec/S/%280000%201%20Apr%202021%29/VALUES/L/1.5/3.5/RANGEEDGES/%5BL%5D//keepgrids/average/Y/-5/40/RANGEEDGES/X/-120/-70/RANGEEDGES/%5BM%5D/average/92/mul/-999/setmissing_value/%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv

DOWNLOADING: [*************************] (35 KB) 0:00:00.410376
URL: https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.CanSIPS-IC3/.FORECAST/.MONTHLY/.prec/S/%280000%201%20Apr%202021%29/VALUES/L/1.5/3.5/RANGEEDGES/%5BL%5D//keepgrids/average/%5BM%5D/average/Y/-5/40/RANGEEDGES/X/-120/-70/RANGEEDGES/92/mul/-999/setmissing_value/%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv

ERROR: No se encontró datos de forecast para el modelo CanSIPSIC3.PRCP
URL: https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/.NASA-GEOSS2S/.FORECAST/.MONTHLY/.prec/%5BM%5D/average/S/%280000%201%20Apr%202021%29/VALUES/L/1.5/3.5/RANGEEDGES/%5BL%5D//keepgrids/average/Y/-5/40/RANGEEDGES/X/-120/-70/RANGEEDGES/92/mul/-999/setmissing_v

#### Perform Analysis 

In [59]:
hcsts, fcsts, skill, pxs, pys = [], [], [], [], []

for i, model_hcst in enumerate(hindcast_data):
    print("Ejecutando análisis del modelo: ", predictor_names[i].split(".")[0])
    if str(MOS).upper() == 'CCA':
        
        # fit CCA model between X & Y and produce real-time forecasts for F 
        cca_h, cca_rtf, cca_s, cca_px, cca_py = cc.canonical_correlation_analysis(model_hcst, Y, \
        F=forecast_data[i], **cpt_args, cpt_kwargs={"interactive": False})

#         fit CCA model again between X & Y, and produce in-sample probabilistic hindcasts 
#         this is using X in place of F, with the year coordinates changed to n+100 years
#         because CPT does not allow you to make forecasts for in-sample data
        cca_h, cca_f, cca_s, cca_px, cca_py = cc.canonical_correlation_analysis(model_hcst, Y, \
        F=ce.redate(model_hcst, yeardelta=48), **cpt_args)
        cca_h = xr.merge([cca_h, ce.redate(cca_f.probabilistic, yeardelta=-48), ce.redate(cca_f.prediction_error_variance, yeardelta=-48)])
        
#         # use the in-sample probabilistic hindcasts to perform probabilistic forecast verification
#         # warning - this produces unrealistically optimistic values 
        cca_pfv = cc.probabilistic_forecast_verification(cca_h.probabilistic, Y, **cpt_args)
        cca_s = xr.merge([cca_s, cca_pfv])

        hcsts.append(cca_h)
        fcsts.append(cca_rtf)
        skill.append(cca_s.where(cca_s > -999, other=np.nan))
        pxs.append(cca_px)
        pys.append(cca_py)
        
    elif str(MOS).upper() == 'PCR':
        
        # fit PCR model between X & Y and produce real-time forecasts for F 
        pcr_h, pcr_rtf, pcr_s, pcr_px = cc.principal_components_regression(model_hcst, Y, F=forecast_data[i], **cpt_args)
        
        # fit PCR model again between X & Y, and produce in-sample probabilistic hindcasts 
        # this is using X in place of F, with the year coordinates changed to n+100 years
        # because CPT does not allow you to make forecasts for in-sample data
        pcr_h, pcr_f, pcr_s, pcr_px = cc.principal_components_regression(model_hcst, Y, F=ce.redate(model_hcst, yeardelta=48), **cpt_args)
        pcr_h = xr.merge([pcr_h, ce.redate(pcr_f.probabilistic, yeardelta=-48), ce.redate(pcr_f.prediction_error_variance, yeardelta=-48)])
        
        # use the in-sample probabilistic hindcasts to perform probabilistic forecast verification
        # warning - this produces unrealistically optimistic values 
        pcr_pfv = cc.probabilistic_forecast_verification(pcr_h.probabilistic, Y, **cpt_args)
        pcr_s = xr.merge([pcr_s, pcr_pfv])
        hcsts.append(pcr_h)
        fcsts.append(pcr_rtf)
        skill.append(pcr_s.where(pcr_s > -999, other=np.nan))
        pxs.append(pcr_px)
    else:
        # simply compute deterministic skill scores of non-corrected ensemble means 
        nomos_skill = cc.deterministic_skill(model_hcst, Y, **cpt_args)
        skill.append(nomos_skill.where(nomos_skill > -999, other=np.nan))
        
    # choose what data to export here (any of the above results data arrays can be saved to netcdf)
    if str(MOS).upper() == 'CCA':
        cca_h.to_netcdf(f"{outputDir}/{predictor_names[i]}_crossvalidated_cca_hindcasts.nc")
        if cca_rtf is not None:
            cca_rtf.to_netcdf(f"{outputDir}/{predictor_names[i]}_realtime_cca_forecasts.nc")
        cca_s.to_netcdf(f"{outputDir}/{predictor_names[i]}_skillscores_cca.nc")
        cca_px.to_netcdf(f"{outputDir}/{predictor_names[i]}_cca_x_spatial_loadings.nc")
        cca_py.to_netcdf(f"{outputDir}/{predictor_names[i]}_cca_y_spatial_loadings.nc")
    elif str(MOS).upper() == 'PCR':
        pcr_h.to_netcdf(f"{outputDir}/{predictor_names[i]}_crossvalidated_pcr_hindcasts.nc")
        pcr_rtf.to_netcdf(f"{outputDir}/{predictor_names[i]}_realtime_pcr_forecasts.nc")
        pcr_s.to_netcdf(f"{outputDir}/{predictor_names[i]}_skillscores_pcr.nc")
        pcr_px.to_netcdf(f"{outputDir}/{predictor_names[i]}_pcr_x_spatial_loadings.nc")
    else: 
        nomos_skill.to_netcdf(f"{outputDir}/{predictor_names[i]}_nomos_skillscores.nc")
        

Ejecutando análisis del modelo:  CCSM4
Ejecutando análisis del modelo:  CanSIPSIC3
Ejecutando análisis del modelo:  GEOSS2S
Ejecutando análisis del modelo:  SPEAR
Ejecutando análisis del modelo:  CFSv2
Ejecutando análisis del modelo:  SEAS5
Ejecutando análisis del modelo:  METEOFRANCE8


CPTError: 
PROCESS STATUS: ALIVE (WILL BE STOPPED)
  last command: '311'
  last message:'Reading data ...
 Reading /home/guillermo/.pycpt_workspace/231af297-0e34-4d0b-840f-edd05154eb36/original_predictand ...
 Reading /home/guillermo/.pycpt_workspace/231af297-0e34-4d0b-840f-edd05154eb36/original_predictor ...
  
 Checking for missing values ...
  
  
                                     Error
  
 ERROR:  The X file contains too many missing series
         /home/guillermo/.pycpt_workspace/231af297-0e34-4d0b-840f-edd05154eb36/original_predictor
 CPT '
